<a href="https://colab.research.google.com/github/JemyHo/NLPCW1/blob/Lihang/cw1/F21NL_CW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O wikitext-filtered-full.zip "https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1"
!wget -O wikitext-filtered-10k.zip "https://www.dropbox.com/scl/fi/ek174r3sg7qjx0aa9atop/wikitext-filtered-10k.zip?rlkey=zy6jqxv6qsc16lr9qm3ki9uhf&dl=1"

--2025-10-19 02:39:13--  https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucff1b788561f2773ee610a23458.dl-eu.dropboxusercontent.com/cd/0/inline/CzgM_M4IzbaQX9BpHUQz9vfJuzK0TLMB3Epbx-vOS_rWNeAFNNoxgZzX3BtlGTiVprbpnW61d0w7vDnpMv-0KKPB2VBmKyYu-Sqk3M7Pz0JkD_G2zylmVB64XQO2t1qMp1W1ZvO7Mn8hW48DNUPJCWyr/file?dl=1# [following]
--2025-10-19 02:39:14--  https://ucff1b788561f2773ee610a23458.dl-eu.dropboxusercontent.com/cd/0/inline/CzgM_M4IzbaQX9BpHUQz9vfJuzK0TLMB3Epbx-vOS_rWNeAFNNoxgZzX3BtlGTiVprbpnW61d0w7vDnpMv-0KKPB2VBmKyYu-Sqk3M7Pz0JkD_G2zylmVB64XQO2t1qMp1W1ZvO7Mn8hW48DNUPJCWyr/file?dl=1
Resolving ucff1b788561f2773ee610a23458.dl-eu.dropboxusercontent.com (ucff1b788561f2773ee610a23458.dl-eu.dropbo

In [2]:
!unzip wikitext-filtered-full.zip
!unzip wikitext-filtered-10k.zip

Archive:  wikitext-filtered-full.zip
   creating: wikitext-filtered-full/
  inflating: wikitext-filtered-full/dataset_info.json  
  inflating: wikitext-filtered-full/state.json  
  inflating: wikitext-filtered-full/data-00000-of-00001.arrow  
Archive:  wikitext-filtered-10k.zip
   creating: wikitext-filtered-10k/
  inflating: wikitext-filtered-10k/dataset_info.json  
  inflating: wikitext-filtered-10k/state.json  
  inflating: wikitext-filtered-10k/data-00000-of-00001.arrow  


In [3]:
!pip install datasets
import datasets

In [4]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 24.9 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset, Dataset

def load_dataset():
  wikitext_small = "wikitext-filtered-10k"
  wikitext_large = "wikitext-filtered-full"

  dataset_small = Dataset.load_from_disk(wikitext_small)
  dataset_large = Dataset.load_from_disk(wikitext_large)
  print("wikitext_small: {} docs, wikitext_large: {} docs".format(len(dataset_small), len(dataset_large)))
  return dataset_small, dataset_large

wikitext_small, wikitext_large = load_dataset()

wikitext_small: 10000 docs, wikitext_large: 859955 docs


In [6]:
import nltk, string, os
from gensim.models import Word2Vec
nltk.download('stopwords')
from nltk.corpus import stopwords

STOP = set(stopwords.words('english'))
print(f"{len(STOP)} English stopwords loaded.")

198 English stopwords loaded.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
#Step 3

def tokenize_by_space(text):
    """
    Minimal preprocessing (as required in spec):
    - lowercase
    - split by spaces
    - strip punctuation
    - remove stopwords and empty tokens
    """
    tokens = []
    for tok in text.lower().split(" "):
        tok = tok.strip(string.punctuation)
        if not tok or tok in STOP:
            continue
        tokens.append(tok)
    return tokens

def prepare_corpus(raw_ds):
    """Convert HuggingFace Dataset to list of token lists."""
    first = raw_ds[0]
    # find the right text field
    if isinstance(first, dict):
        key = [k for k in first.keys() if isinstance(first[k], str)][0]
        docs = [ex[key] for ex in raw_ds]
    else:
        docs = list(raw_ds)
    return [tokenize_by_space(doc) for doc in docs if doc.strip()]

In [8]:
tokens_small = prepare_corpus(wikitext_small)
# tokens_large = prepare_corpus(wikitext_large)

print("Docs (small):", len(tokens_small))
# print("Docs (large):", len(tokens_large))
print("Example tokens:", tokens_small[0][:20])


Docs (small): 10000
Example tokens: ['senjō', 'valkyria', '3', 'unrecorded', 'chronicles', 'japanese', '戦場のヴァルキュリア3', 'lit', 'valkyria', 'battlefield', '3', 'commonly', 'referred', 'valkyria', 'chronicles', 'iii', 'outside', 'japan', 'tactical', 'role']


In [9]:
SEED = 42
WORKERS = os.cpu_count()

def train_word2vec(sentences, name, vector_size=50, window=5, min_count=5, epochs=5, sg=1):
    print(f"▶ Training {name} | vec={vector_size}, win={window}, min={min_count}, ep={epochs}, sg={sg}")
    model = Word2Vec(
        sentences=sentences,
        vector_size=vector_size,
        window=window,
        min_count=min_count,
        sg=sg,      # 0 = CBOW, 1 = Skip-gram
        workers=WORKERS,
        seed=SEED
    )
    model.train(sentences, total_examples=len(sentences), epochs=epochs)
    print("Vocab size:", len(model.wv))
    model.save(f"w2v_{name}.model")
    print(f" Saved model: w2v_{name}.model\n")
    return model

w2v_small = train_word2vec(tokens_small, "small")
# w2v_large = train_word2vec(tokens_large, "large")


▶ Training small | vec=50, win=5, min=5, ep=5, sg=0


Vocab size: 14554
✅ Saved model: w2v_small.model



In [10]:
import nltk, string, os
from gensim.models import Word2Vec
nltk.download('stopwords')
from nltk.corpus import stopwords

STOP = set(stopwords.words('english'))
print(f"{len(STOP)} English stopwords loaded.")


198 English stopwords loaded.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec


# Load both trained Word2Vec models

w2v_small = Word2Vec.load("w2v_small.model")
# w2v_large = Word2Vec.load("w2v_large.model")
print("Models loaded")
print("small vocab:", len(w2v_small.wv), )#"| large vocab:", len(w2v_large.wv)

Models loaded
small vocab: 14554


In [13]:
# Load WordSim-353 CSV from Google Drive

file_path = '/content/drive/My Drive/wordsim353/combined.csv'

try:
    wordsim_df = pd.read_csv(file_path)
    print("WordSim-353 data loaded successfully.")
    display(wordsim_df.head())
except FileNotFoundError:
    print(f"File not found at {file_path}")
except Exception as e:
    print(f"Error loading file: {e}")

print("Total pairs:", len(wordsim_df))


WordSim-353 data loaded successfully.


,Word 1,Word 2,Human (mean)
0,love,sex,6.77
1,tiger,cat,7.35
2,tiger,tiger,10.00
3,book,paper,7.46
4,computer,keyboard,7.62


Total pairs: 353


In [14]:
# Helper: compute cosine safely

def compute_cosine(model, w1, w2):
    w1, w2 = w1.lower(), w2.lower()
    try:
        return model.wv.similarity(w1, w2)
    except KeyError:
        return np.nan   # skip missing words

# Compute cosine for the 4 required pairs

pairs = [
    ("plane", "car"),
    ("planet", "sun"),
    ("cup", "article"),
    ("sugar", "approach")
]

print("\n---- wikitext_small ----")
for a, b in pairs:
    print(f"{a:8s}/{b:8s} → {compute_cosine(w2v_small, a, b):.4f}")

# print("\n---- wikitext_large ----")   ####
# for a, b in pairs:   ###
   # print(f"{a:8s}/{b:8s} → {compute_cosine(w2v_large, a, b):.4f}")   #####


# Prepare for full 353-pair evaluation (Step 4)

def evaluate_all(model, df):
    cosines = []
    for _, row in df.iterrows():
        w1, w2 = row['Word 1'], row['Word 2']
        cos = compute_cosine(model, w1, w2)
        cosines.append(cos)
    df[f'{model}'] = cosines
    return cosines

# compute for both models (optional preview)
wordsim_df['cosine_small'] = [
    compute_cosine(w2v_small, w1, w2) for w1, w2 in zip(wordsim_df['Word 1'], wordsim_df['Word 2'])
]
# wordsim_df['cosine_large'] = [
   # compute_cosine(w2v_large, w1, w2) for w1, w2 in zip(wordsim_df['Word 1'], wordsim_df['Word 2'])
# ]

display(wordsim_df.head())
print("✅ Cosine columns added – ready for Step 4 (Spearman).")





---- wikitext_small ----
plane   /car      → 0.6788
planet  /sun      → 0.8853
cup     /article  → 0.2885
sugar   /approach → 0.8050


,Word 1,Word 2,Human (mean),cosine_small
0,love,sex,6.77,0.830847
1,tiger,cat,7.35,0.869717
2,tiger,tiger,10.00,1.000000
3,book,paper,7.46,0.693030
4,computer,keyboard,7.62,0.849133


✅ Cosine columns added – ready for Step 4 (Spearman).


In [15]:
# Step 4
from scipy.stats import spearmanr

# Compute Spearman rank-order correlation
rho_small, p_small = spearmanr(wordsim_df['Human (mean)'],
                               wordsim_df['cosine_small'],
                               nan_policy='omit')

#rho_large, p_large = spearmanr(wordsim_df['Human (mean)'],
                             #  wordsim_df['cosine_large'],
                             #  nan_policy='omit')

print(f"Spearman correlation (wikitext_small): {rho_small:.4f} (p={p_small:.4e})")
#print(f"Spearman correlation (wikitext_large): {rho_large:.4f} (p={p_large:.4e})")


Spearman correlation (wikitext_small): 0.1517 (p=1.4173e-02)


In [20]:
#Step 5 (if u are the first time to download the api, just run this cell, remember to clear all the ## before run)
import gensim.downloader as api

# Save it permanently to Drive
## model.save("/content/drive/MyDrive/word2vec-google-news-300.model")
## print("Model saved to Google Drive!")

# Load model
model = api.load("word2vec-google-news-300")
print("Model loaded!")



#"""Once u done download the api model, it should be save to ur google drive"""
#"""Then , put the ## back to the code"""
#"""And so when u run again, it just only call the api from your google drive, no need download again"""



Model loaded!


In [22]:
from scipy.stats import spearmanr
import numpy as np
from gensim.models import KeyedVectors

google_news_model = KeyedVectors.load("/content/drive/MyDrive/word2vec-google-news-300.model", mmap='r')
print("Loaded Google News model from Drive (no download needed!)")

# Safe cosine function for this KeyedVectors model
def compute_cosine_pretrained(model, w1, w2):
    w1, w2 = w1.lower(), w2.lower()
    try:
        return model.similarity(w1, w2)
    except KeyError:
        return np.nan

# Compute cosine for all pairs
wordsim_df['cosine_google'] = [
    compute_cosine_pretrained(google_news_model, w1, w2)
    for w1, w2 in zip(wordsim_df['Word 1'], wordsim_df['Word 2'])
]

# Calculate Spearman correlation
rho_google, p_google = spearmanr(wordsim_df['Human (mean)'], wordsim_df['cosine_google'], nan_policy='omit')

print(f"Spearman correlation (Google News Word2Vec): {rho_google:.4f} (p={p_google:.4e})")


Loaded Google News model from Drive (no download needed!)
Spearman correlation (Google News Word2Vec): 0.6941 (p=1.2875e-51)


In [23]:
# STEP 6

def train_and_evaluate(tokens, name, **params):
    """Train model with given parameters and compute Spearman correlation."""
    model = Word2Vec(sentences=tokens, workers=os.cpu_count(), seed=42, **params)
    model.train(tokens, total_examples=len(tokens), epochs=params.get("epochs", 5))

    # Compute cosine for all 353 pairs
    df = wordsim_df.copy()
    df['cosine'] = [
        compute_cosine(model, w1, w2) for w1, w2 in zip(df['Word 1'], df['Word 2'])
    ]
    rho, _ = spearmanr(df['Human (mean)'], df['cosine'], nan_policy='omit')
    print(f"{name:15s} → Spearman ρ = {rho:.4f}")
    return rho

results = {}

# Example experiments
results['baseline'] = train_and_evaluate(tokens_small, "baseline",
                                         vector_size=50, window=5, min_count=5, epochs=5, sg=0)

results['skipgram'] = train_and_evaluate(tokens_small, "skipgram",
                                         vector_size=50, window=5, min_count=5, epochs=5, sg=1)

results['bigger'] = train_and_evaluate(tokens_small, "bigger",
                                       vector_size=100, window=5, min_count=5, epochs=10, sg=1)

pd.DataFrame(list(results.items()), columns=["Model", "Spearman ρ"])


baseline        → Spearman ρ = 0.1346


skipgram        → Spearman ρ = 0.3105


bigger          → Spearman ρ = 0.4493


,Model,Spearman ρ
0,baseline,0.134599
1,skipgram,0.310459
2,bigger,0.449261


In [24]:
#step 7

from gensim.models import KeyedVectors


# load from Drive (fast) — change path if needed
google_news = KeyedVectors.load("/content/drive/MyDrive/word2vec-google-news-300.model", mmap='r')

def normalize_variants(w):
    """Return candidate surface forms for GoogleNews vocab (case & underscore)."""
    cands = {w, w.lower(), w.capitalize(), w.title(), w.replace(" ", "_"),
             w.lower().replace(" ", "_")}
    return list(cands)

def in_vocab(model, w):
    return w in model.key_to_index

def pick_vocab_form(model, w):
    for v in normalize_variants(w):
        if in_vocab(model, v):
            return v
    return None  # not found

def topk_analogy(model, a, b, c, topn=5, exclude=None):
    """
    Solve: a : b :: c : ?
    i.e., vector = b - a + c, return top-n.
    Tries smart variants for vocab. Excludes source words by default.
    """
    a_ = pick_vocab_form(model, a)
    b_ = pick_vocab_form(model, b)
    c_ = pick_vocab_form(model, c)
    if not all([a_, b_, c_]):
        return {"query": (a,b,c), "resolved": (a_,b_,c_), "answers": [], "note": "OOV encountered"}

    neg = [a_]
    pos = [b_, c_]
    res = model.most_similar(positive=pos, negative=neg, topn=topn+10)  # extra buffer
    # filter out trivial echoes of inputs
    bad = set([x for x in [a_, b_, c_] if x]) | set(normalize_variants(a) + normalize_variants(b) + normalize_variants(c))
    answers = [(w, float(s)) for (w,s) in res if w not in bad][:topn]
    return {"query": (a,b,c), "resolved": (a_,b_,c_), "answers": answers, "note": ""}


In [27]:
queries_g = [
    ("man", "woman", "king"),                # → expect ~"queen"
    ("Athens", "Greece", "Rome"),            # → expect ~"Italy"
    ("reading", "read", "playing"),          # → expect ~"play"
    ("Greece", "souvlaki", "Italy"),         # → expect ~"pizza"/"pasta"
    ("airplane", "propeller", "car"),        # → expect ~"engine"/"motor"
]

for a,b,c in queries_g:
    out = topk_analogy(google_news, a,b,c, topn=5)
    print(f"{a} : {b} :: {c} : ?")
    print(" resolved tokens:", out["resolved"])
    if out["answers"]:
        for i,(w,s) in enumerate(out["answers"],1):
            print(f"  {i}. {w:20s}  ({s:.3f})")
    else:
        print("  [OOV or no results]", out["note"])
    print()


man : woman :: king : ?
 resolved tokens: ('Man', 'woman', 'king')
  1. queen                 (0.600)
  2. monarch               (0.581)
  3. prince                (0.524)
  4. princess              (0.512)
  5. crown_prince          (0.479)

Athens : Greece :: Rome : ?
 resolved tokens: ('Athens', 'greece', 'rome')
  1. pablo                 (0.621)
  2. samuel                (0.621)
  3. omar                  (0.619)
  4. cro                   (0.615)
  5. zim                   (0.613)

reading : read :: playing : ?
 resolved tokens: ('Reading', 'Read', 'playing')
  1. played                (0.524)
  2. play                  (0.472)
  3. playin                (0.462)
  4. See                   (0.419)
  5. practicing            (0.400)

Greece : souvlaki :: Italy : ?
 resolved tokens: ('greece', 'souvlaki', 'Italy')
  1. Italian               (0.522)
  2. Pasticceria           (0.507)
  3. cannolis              (0.502)
  4. Napoletana            (0.496)
  5. Undici                (0.

In [26]:
queries_h = [
    ("man", "woman", "computer programmer"),
    ("man", "woman", "superstar"),
    ("man", "woman", "guitarist"),
    ("man", "woman", "boss"),
]

for a,b,c in queries_h:
    out = topk_analogy(google_news, a,b,c, topn=5)
    print(f"{a} : {b} :: {c} : ?")
    print(" resolved tokens:", out["resolved"])
    if out["answers"]:
        for i,(w,s) in enumerate(out["answers"],1):
            print(f"  {i}. {w:25s} ({s:.3f})")
    else:
        print("  [OOV or no results]", out["note"])
    print()


man : woman :: computer programmer : ?
 resolved tokens: ('Man', 'woman', 'computer_programmer')
  1. schoolteacher             (0.524)
  2. homemaker                 (0.516)
  3. businesswoman             (0.483)
  4. housewife                 (0.481)
  5. divorcee                  (0.475)

man : woman :: superstar : ?
 resolved tokens: ('Man', 'woman', 'superstar')
  1. megastar                  (0.577)
  2. singing_sensation         (0.511)
  3. diva                      (0.511)
  4. songstress                (0.488)
  5. pop_diva                  (0.487)

man : woman :: guitarist : ?
 resolved tokens: ('Man', 'woman', 'Guitarist')
  1. Vocalist                  (0.541)
  2. vocalist                  (0.527)
  3. Singer_songwriter         (0.519)
  4. drummer                   (0.510)
  5. Bassist                   (0.508)

man : woman :: boss : ?
 resolved tokens: ('Man', 'woman', 'boss')
  1. lady                      (0.471)
  2. husband                   (0.450)
  3. receptionis